In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os

os.chdir('../../test_data/system_tests/multicond')

In [3]:
RESULTS_DIR = "results"

In [26]:
import alphaquant.utils.utils as utils
import pandas as pd

class MedianRefResultLoader():
    def __init__(self, results_dir):
        self._results_dir = results_dir
        self.condpair2tree = {}

        self._define_condpair2tree()
    
    def _define_condpair2tree(self):
        self.condpair2tree = utils.read_all_trees_in_results_folder(self._results_dir)
        self._remove_all_condpairs_w_no_medianref()
    
    def _remove_all_condpairs_w_no_medianref(self):
        keys_to_remove = [x for x in self.condpair2tree.keys() if "median_reference" not in x]
        for key in keys_to_remove:
            del self.condpair2tree[key]
    
    

class MedianRefConditionCombiner():
    def __init__(self, condpair2tree):
        self._condpair2tree = condpair2tree

        self.protein2nodes = {}
        self.peptide_resolved_proteoform_df = None

        self._define_protein2nodes()      
        self._define_combined_dataframe()

    def _define_protein2nodes(self):
        for tree in self._condpair2tree.values():
            cond = tree.name[0]
            for protnode in tree.children:
                self._extend_protein2nodes(protnode, cond)

    def _extend_protein2nodes(self, protnode, cond):
        protnode.cond = cond
        if protnode.name not in self.protein2nodes:
            self.protein2nodes[protnode.name] = []
        self.protein2nodes[protnode.name].append(protnode)

    def _define_combined_dataframe(self):
        list_of_proteoform_dfs = []
        for protein, nodes in self.protein2nodes.items():
            proteoform_df = ProteoformConditionAligner(nodes).proteoform_df
            list_of_proteoform_dfs.append(proteoform_df)
        self.peptide_resolved_proteoform_df = pd.concat(list_of_proteoform_dfs)
    


In [32]:

df_reformatter = CombinedProteoformDfFormatter(peptide_resolved_proteoform_df)

display(peptide_resolved_proteoform_df)
display(df_reformatter.proteoform_df)
display(df_reformatter.protein_df_average)
display(df_reformatter.protein_df_pform0)


NameError: name 'peptide_resolved_proteoform_df' is not defined

In [ ]:


def ends_with_zero(x):
    return x.endswith('0')

count_ends_with_zero = peptide_resolved_proteoform_df[peptide_resolved_proteoform_df['proteoform_id'].apply(ends_with_zero)].groupby('protein_name').size()

total_count = peptide_resolved_proteoform_df.groupby('protein_name').size()

fraction_ends_with_zero = count_ends_with_zero / total_count

display(fraction_ends_with_zero)

import matplotlib.pyplot as plt

plt.hist(fraction_ends_with_zero, cumulative=False, density=True)


